<a href="https://colab.research.google.com/github/mjgpinheiro/Physics_models/blob/main/Stock_Trading_Bot_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import datetime
import yfinance as yf
import numpy as np
import pandas as pd
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Gather Data
symbol = "AAPL"
start_date = "2010-01-01"
end_date = datetime.datetime.now().strftime("%Y-%m-%d")
df = yf.download(symbol, start=start_date, end=end_date, interval="1d")

# Preprocess Data
scaler = MinMaxScaler()
df["Close"] = scaler.fit_transform(df["Close"].values.reshape(-1, 1))

# Split Data
train_size = int(len(df) * 0.8)
train_data = df[:train_size]
test_data = df[train_size:]

def create_sequences(data, seq_length):
    sequences = []
    labels = []
    for i in range(len(data) - seq_length):
        sequence = data[i : i + seq_length]
        label = data[i + seq_length]
        sequences.append(sequence)
        labels.append(label)
    return np.array(sequences), np.array(labels)

seq_length = 60
train_x, train_y = create_sequences(train_data["Close"], seq_length)
test_x, test_y = create_sequences(test_data["Close"], seq_length)

# Build Model
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(train_x.shape[1], 1)))
model.add(LSTM(units=50))
model.add(Dense(units=1))
model.compile(optimizer="adam", loss="mean_squared_error")

# Train Model
model.fit(train_x, train_y, epochs=10, batch_size=32)

# Make Predictions
predictions = model.predict(test_x)
predictions = scaler.inverse_transform(predictions)

# Evaluate Model
mse = mean_squared_error(predictions, scaler.inverse_transform(test_y.reshape(-1, 1)))
mae = mean_absolute_error(predictions, scaler.inverse_transform(test_y.reshape(-1, 1)))
print("Mean Squared Error:", mse)
print("Mean Absolute Error:", mae)

# Save Model
model.save("stock_trading_bot_model.keras")

# Make a prediction for the next day
next_day_sequence = np.expand_dims(test_x[-1], axis=0)
next_day_prediction = model.predict(next_day_sequence)
next_day_price = scaler.inverse_transform(next_day_prediction)[0][0]

# Print the predicted price for the next day
print("Predicted price for the next day:", next_day_price)

# Determine the action based on the predicted price
if next_day_price > df["Close"].iloc[-1]:
    print("Action: Buy")
elif next_day_price < df["Close"].iloc[-1]:
    print("Action: Sell")
else:
    print("Action: Hold")

# Create a baseline model that predicts the last known price as the next day's price
baseline_predictions = np.roll(test_x[:,-1], -1)  # Shift the last known prices one step forward
baseline_mse = mean_squared_error(test_y, baseline_predictions)
baseline_mae = mean_absolute_error(test_y, baseline_predictions)

# Calculate the Mean Squared Error (MSE) and Mean Absolute Error (MAE) for the LSTM model
lstm_mse = mean_squared_error(test_y, predictions)
lstm_mae = mean_absolute_error(test_y, predictions)

# Print the comparison results
print("\nBaseline Model:")
print("MSE:", baseline_mse)
print("MAE:", baseline_mae)

print("\nLSTM Model:")
print("MSE:", lstm_mse)
print("MAE:", lstm_mae)

# Print the comparison results
print("Baseline Model:")
print("MSE:", baseline_mse)
print("MAE:", baseline_mae)

print("\nLSTM Model:")
print("MSE:", lstm_mse)
print("MAE:", lstm_mae)

# Compare the LSTM model's performance with the baseline model's performance
if lstm_mse < baseline_mse and lstm_mae < baseline_mae:
    print("\nConclusion: The LSTM model's performance is better than the baseline model. Consider using LSTM predictions.")
elif lstm_mse > baseline_mse and lstm_mae > baseline_mae:
    print("\nConclusion: The baseline model's performance is better than the LSTM model. Caution advised.")
else:
    print("\nConclusion: The LSTM model's performance is similar to the baseline model. Use additional analysis for decision-making.")


[*********************100%%**********************]  1 of 1 completed
Epoch 1/10
84/84 [==============================] - 9s 51ms/step - loss: 0.0015
Epoch 2/10
84/84 [==============================] - 5s 59ms/step - loss: 1.1668e-04
Epoch 3/10
84/84 [==============================] - 5s 61ms/step - loss: 1.0795e-04
Epoch 4/10
84/84 [==============================] - 4s 52ms/step - loss: 1.0121e-04
Epoch 5/10
84/84 [==============================] - 6s 68ms/step - loss: 9.7070e-05
Epoch 6/10
84/84 [==============================] - 4s 53ms/step - loss: 1.1162e-04
Epoch 7/10
84/84 [==============================] - 4s 53ms/step - loss: 9.0441e-05
Epoch 8/10
84/84 [==============================] - 6s 69ms/step - loss: 7.7740e-05
Epoch 9/10
84/84 [==============================] - 4s 51ms/step - loss: 8.2881e-05
Epoch 10/10
20/20 [==============================] - 2s 29ms/step
Mean Squared Error: 20.93761940264064
Mean Absolute Error: 3.5444252586060543
1/1 [==============================